In [1]:
import numpy as np
import pandas as pd
import numpy as np
import os
import sys
from spaces import *
import gtda
import matplotlib.pyplot as plt
from pathlib import Path #auxilliary module
from gtda.plotting import plot_diagram
from gtda.homology import VietorisRipsPersistence
from gtda.plotting import plot_diagram
from gtda.diagrams import PersistenceEntropy
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split


In [2]:
sys.path.append("C:\\Users\\Joe\\Documents\\University\\4th Year\\symmetric_tda")

In [3]:
def extract_key_vectors(diagrams, n):
    key_vectors = [None] * 4

    for batch in diagrams:
        for vector in batch:
            V1, V2, V3 = vector # v1 = birth, v2 = death, v3 = dimension
            
            # First key vector: V[0] is the vector with the latest death for which v3 = n - 1
            if V3 == n - 1:
                if key_vectors[0] is None or V2 > key_vectors[0][1]:
                    key_vectors[0] = vector

            # Second key vector: V[1] is the vector with the earliest birth for which v3 = n
            if V3 == n:
                if key_vectors[1] is None or V1 < key_vectors[1][0]:
                    key_vectors[1] = vector


            # Third key vector: V[2] is the vector with the latest death for which v3 = n
            if V3 == n:
                if key_vectors[2] is None or V2 > key_vectors[2][1]:
                    key_vectors[2] = vector

            # Fourth key vector: V[3] is the vector with the earliest birth for which v3 = n + 1
            if V3 == n + 1:
                if key_vectors[3] is None or V1 < key_vectors[3][0]:
                    key_vectors[3] = vector

    return key_vectors

In [20]:
def extract_key_values(diagrams, n):
    key_values_per_batch = []

    for batch in diagrams:
        Z0 = batch
        d = n
        [a0, a1, a2, a3] = [None] * 4
        # Efficiently calculate the key values
        a0 = Z0[Z0[:,2] < d][:,1].max()
        a1 = Z0[Z0[:,2] == d][:,0].min()
        a2 = Z0[Z0[:,2] == d][:,1].max()
        a3 = Z0[Z0[:,2] > d][:,0].min()
        # Append the key values as a vector
        key_values_per_batch.append([a0, a1, a2, a3])

    return key_values_per_batch

In [4]:

class Torus(Space): #regular torus
    def __init__(self, R, r):
        """
        Create a torus with major radius R and minor radius r.
        The embedding dimension is 3 (in 3D space).
        """
        self.R = R  # Major radius
        self.r = r  # Minor radius
        super().__init__('Torus', 3, 2)

    def sample(self, sample_size, num_samples=1):
        """
        Returns a numpy matrix of shape n x edim, where each row is a
        random point on the torus. To do this, we generate random angles
        theta and phi, and then compute the corresponding points on the torus.
        """
        theta = np.random.uniform(0, 2 * np.pi, sample_size * num_samples)
        phi = np.random.uniform(0, 2 * np.pi, sample_size * num_samples)

        x = (self.R + self.r * np.cos(phi)) * np.cos(theta)
        y = (self.R + self.r * np.cos(phi)) * np.sin(theta)
        z = self.r * np.sin(phi)

        points = np.vstack((x, y, z)).T
        self.points = points.reshape(num_samples, sample_size, self.edim)
        return self.points

In [5]:
class NTorus(Space):
    def __init__(self, n):
        """
        Create an n-dimensional torus. The embedding dimension is 2n.
        """
        self.n = n
        super().__init__('T^{' + str(n) + '}', 2 * n, n)

    def sample(self, sample_size, num_samples=1):
        """
        Returns a numpy matrix of shape (num_samples, sample_size, 2n), where each row is a
        random point on the n-dimensional torus. To do this, we generate random angles
        theta_i uniformly distributed between 0 and 2*pi for each dimension.
        """
        theta = np.random.uniform(0, 2 * np.pi, (sample_size * num_samples, self.n))

        # Compute the corresponding points in the embedding space
        points = np.hstack((np.cos(theta), np.sin(theta)))

        self.points = points.reshape(num_samples, sample_size, self.edim)
        return self.points

In [6]:
T = Space.fit(NTorus(3), 70, 1)

In [5]:
results = []
#plot_diagram(S[0])
k = 45 # number of points sampled
p = 1 # number of batches
for n in range(2, 6):

    V = extract_key_vectors(Space.fit(Sphere(n), k, p), n)

    # Calculate the difference in distance between the birth and death of key vector 1 and key vector 0, respectively
    differencebefore = V[1][0] - V[0][1]
    # Calculate the difference in distance between the birth and death of key vector 3 and key vector 2, respectively
    differenceafter = V[3][0] - V[2][1]
    rng = V[2][1] - V[1][0] #range in which the homology is correct
    # Append the result to the list
    results.append({'Dimension of Sphere': n, 'Difference before': differencebefore, 'Difference after': differenceafter, 'Range': rng})

# Convert the results to a DataFrame
results_df = pd.DataFrame(results)

# Display the results
print(results_df)

   Dimension of Sphere  Difference before  Difference after     Range
0                    2           0.064934          0.031202  0.491755
1                    3           0.066893          0.007565  0.075539
2                    4          -0.290804          0.000000  0.365355
3                    5          -0.101817          0.003265  0.185310


In [ ]:
results = []
#plot_diagram(S[0])
k = 105 # number of points sampled
p = 1 # number of batches
for n in range(2, 4):

    V = extract_key_values(Space.fit(Sphere(n), k, p), n)
    # Calculate the difference in distance between the birth and death of key vector 1 and key vector 0, respectively
    differencebefore = V[0][1] - V[0][0]
    # Calculate the difference in distance between the birth and death of key vector 3 and key vector 2, respectively
    differenceafter = V[0][3] - V[0][2]
    rng = V[0][2] - V[0][1] #range in which the homology is correct
    # Append the result to the list
    results.append({'Dimension of Sphere': n, 'Difference before': differencebefore, 'Difference after': differenceafter, 'Range': rng})

# Convert the results to a DataFrame
results_df = pd.DataFrame(results)

# Display the results
print(results_df)

   Dimension of Sphere  Difference before  Difference after     Range
0                    2           0.324573          0.011523  0.626891
1                    3           0.001210          0.003818  0.484681


In [34]:
Q = Space.fit(Sphere(2), 245, 1)

In [35]:
Sval = extract_key_values(Q, 2)
print(Sval)

[[0.6571176648139954, 0.635789692401886, 1.6428406238555908, 1.6518667936325073]]


In [36]:
Svect = extract_key_vectors(Q, 2)
print(Svect)

[array([0.30441949, 0.65711766, 1.        ]), array([0.63578969, 0.63677025, 2.        ]), array([0.6817475 , 1.64284062, 2.        ]), array([1.65186679, 1.69560254, 3.        ])]
